# Konnektor - Building Ligand Networks

this is Konnektor, which you can use to construct different types of Networks.

In [ ]:
import numpy as np
from rdkit import Chem

try:
    from kartograf import KartografAtomMapper, MappingRMSDScorer
except ModuleNotFoundError:
    !{sys.executable} -m pip install git+https://github.com/OpenFreeEnergy/kartograf.git
    from kartograf import KartografAtomMapper, MappingRMSDScorer


import logging
from kartograf.atom_mapper import logger
logger.setLevel(logging.ERROR)


In [ ]:
from kartograf import SmallMoleculeComponent
from konnektor.visualization import draw_ligand_network, draw_network_widget

mapper = KartografAtomMapper()
scorer = MappingRMSDScorer()

from konnektor.data import get_hif2a_ligands
compounds = get_hif2a_ligands()
#rdmols = list(Chem.SDMolSupplier(benzenes_sdf_path, removeHs=False))
#compounds = [SmallMoleculeComponent.from_rdkit(m) for m in rdmols]
#compounds = list(filter(lambda x: not x.name in ["lig_2", "lig_3", "lig_4", "lig_7"], compounds))
Chem.Draw.MolsToGridImage ([c.to_rdkit() for c in compounds], )

# Radial Network Layout

In [ ]:
from konnektor.network_planners import StarNetworkGenerator
ligand_network_planner = StarNetworkGenerator(mapper=KartografAtomMapper(), scorer=scorer)

In [ ]:
radial_network = ligand_network_planner(compounds)
radial_network.name="Star Network"
radial_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(radial_network);

S## Starry Sky Network Layout

In [ ]:
from konnektor.network_planners import StarrySkyNetworkGenerator

ligand_network_planner = StarrySkyNetworkGenerator(mapper=KartografAtomMapper(), 
                                                       scorer=scorer)

In [ ]:
starry_sky_network = ligand_network_planner(compounds)
starry_sky_network.name="Starry Sky Network"
starry_sky_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(starry_sky_network);

# Minimal Spanning Tree

In [ ]:
from konnektor.network_planners import MinimalSpanningTreeNetworkGenerator

ligand_network_planner = MinimalSpanningTreeNetworkGenerator(mapper=KartografAtomMapper(), scorer=scorer)


In [ ]:
mst_network = ligand_network_planner(compounds)
mst_network.name="Min Spanning Tree Network"
mst_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(mst_network, layout="breadthfirst");

# Redundant MST

In [ ]:
from konnektor.network_planners import RedundantMinimalSpanningTreeNetworkGenerator

ligand_network_planner = (RedundantMinimalSpanningTreeNetworkGenerator
                          (mapper=KartografAtomMapper(), scorer=scorer,
                           n_redundancy=2))


In [ ]:
rmst_network = ligand_network_planner(compounds)
rmst_network.name="Redundant Min Spanning Tree Network"
rmst_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(rmst_network);

# Cyclo Graphs

In [ ]:
from konnektor.network_planners import CyclicNetworkGenerator
ligand_network_planner = CyclicNetworkGenerator(mapper=KartografAtomMapper(), scorer=scorer,
                                                    cycle_sizes=3, node_present_in_cycles=2)

In [ ]:
cyclic_network = ligand_network_planner.generate_ligand_network(compounds)
cyclic_network.name="Cyclic Network"
cyclic_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(cyclic_network);

# Maximally connected network

In [ ]:
import logging
from konnektor.network_planners import MaximalNetworkGenerator
from kartograf.atom_mapper import logger
logger.setLevel(logging.ERROR)

ligand_network_planner = MaximalNetworkGenerator(mapper=KartografAtomMapper(), scorer=scorer)
ligand_network_planner.nprocesses = 3


In [ ]:
len(compounds)*len(compounds)//2

In [ ]:
max_network = ligand_network_planner(compounds)
max_network.name = "Maximal Network"
max_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(max_network, layout="concentric");

# Diversity Cluster Network

In [ ]:
from konnektor.network_planners import ClusteredNetworkGenerator
from konnektor.network_planners import CyclicNetworkGenerator
from konnektor.network_planners import MstConcatenator

sub_network_planner = CyclicNetworkGenerator(mapper=KartografAtomMapper(), scorer=scorer)
network_concatenator = MstConcatenator(mapper=KartografAtomMapper(), scorer=scorer)
ligand_network_planner = ClusteredNetworkGenerator(sub_network_planners=sub_network_planner, concatenator=network_concatenator)

In [ ]:
div_network = ligand_network_planner(compounds)
div_network.name = "Diversity Clustering Network"
div_network

In [ ]:
# NBVAL_SKIP
draw_network_widget(div_network);

# Summary

In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=[9*4,2*9])
axes= np.array(axes).flat

fs = 22
for ax, net in zip(axes, [radial_network, starry_sky_network, 
                          mst_network, rmst_network,
                          cyclic_network, div_network,
                          max_network]):
    draw_ligand_network(network=net, title=net.name, ax=ax, node_size=1500)
    ax.axis("off")
axes[-1].axis("off")


In [ ]:
# NBVAL_SKIP
fig.savefig("../.img/network_layouts.png", dpi=400)